In [187]:
import pandas as pd
import uuid
import numpy as np
from StringIO import StringIO
from toolz import thread_first, thread_last, pipe
from pandas import DataFrame as df

In [188]:
def mapdict(f,d):
    return [f(k,v) for k,v in d.iteritems()]

def generate_sid():
    return str(uuid.uuid4()).split('-')[-1]

def create_well_df(cell_dict):
    return thread_last(well_dict,
                       (mapdict,lambda k,v: {"Cell Type":k,"Well Name":v}),
                       (map, df),
                       pd.concat)

def create_plate_df(well_df,plate_info, plate_name):
    x = well_df.copy()
    x['Plate Name'] = plate_name
    x['Condition'] = plate_info + ' ' + x['Cell Type']
    x['Plate ID'] = generate_sid()
    return x.drop('Cell Type',axis=1)

In [189]:
conditions = [('PML ssC w RNase A',24),
              ('PML APB w RNase A',25),
              ('PML ssC wo RNase A',26),
              ('PML IF only control no EtOH dehydrat',27),
              ('PML ssC control no FISH',28),
              ('IgG APB w LNA probe before denature',29),
              ('IgG APB w LNA probe after denature',30),
              ('IgG ssC w LNA probe no denature',31)]

cell_dict = {"U2OS": ['A01','A02','B01','B02'],
             "143B": ['A03','A04','B03','B04']}

date = '06-19-2015'

In [184]:
well_df = create_well_df(cell_dict)
condition_lookup = pd.concat([create_plate_df(well_df,a,b) for a,b in conditions])
condition_lookup['Date'] = date

In [185]:
condition_lookup.head()

,Well Name,Plate Name,Condition,Plate ID,Date
0,A03,24,PML ssC w RNase A 143B,2f26762ed8d5,06-19-2015
1,A04,24,PML ssC w RNase A 143B,2f26762ed8d5,06-19-2015
2,B03,24,PML ssC w RNase A 143B,2f26762ed8d5,06-19-2015
3,B04,24,PML ssC w RNase A 143B,2f26762ed8d5,06-19-2015
0,A01,24,PML ssC w RNase A U2OS,2f26762ed8d5,06-19-2015


In [186]:
path = '/notebooks/moldev-data/original/06-19-2015/6.19.15 All ssC data - Jonah Simon.txt'
f = open(path).read()
f2 = f.split('ATF')[1:]
data = pd.concat([pd.read_csv(StringIO(my_f),delimiter='\t',skiprows=4) for my_f in f2])